# Project action
- modify plotBtn_clicked() to take annotated timestamps

In [5]:
from PyQt5 import QtGui
import pyqtgraph as pg
import pandas as pd
import numpy as np
from itertools import product
import os

C:\ProgramData\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
## Always start by initializing Qt (only once per application)
app = QtGui.QApplication([])

In [7]:
class AccViewer(QtGui.QDialog):
    def __init__(self, parent=None):
        super(AccViewer,self).__init__(parent)
        
        self.WindowRange = 300 # time (s) to add to both sides of acc signal
        self.DataPath = os.path.join('//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation','ClinicVisitACC')
        self.SavePath = os.path.join('//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation','TaskAcc')
        
        ## Load timestamp data
        Labels = pd.read_hdf('//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\\form509.h5')
        Questions = pd.read_sas('//FS2.smpp.local\RTO\CIS-PD MUSC\datadictionary.sas7bdat')
        Questions = Questions[Questions.zFormID == 509.0]
        Questions = Questions.drop(columns = list(set(Questions.columns)-set(['zFieldNm','zFieldNb'])))
        Questions.zFieldNm = Questions.zFieldNm.apply(
            lambda x: x.replace(b'\x92',b"'").replace(b'\x97',b'--').decode() if isinstance(x, bytes) else x)
        Questions.zFieldNb = Questions.zFieldNb.apply(
            lambda x: x.replace(b'\x92',b"'").replace(b'\x97',b'--').decode() if isinstance(x, bytes) else x)
        
        ## Get list of available subjects
        S = [s for s in os.listdir(self.DataPath) if os.path.isfile(os.path.join(self.DataPath,s))]
        
        ## Class variables to use in widget functions
        self.questions = Questions
        self.timestamps = Labels
        self.subjList = np.unique([s[0:4] for s in S])
        self.taskInd = 0
        self.taskList = np.array(['Shaking', 'Standing', 'Walking', 'Walking while counting', 'Finger to nose--right hand', 
                                  'Finger to nose--left hand', 'Alternating right hand movements', 
                                  'Alternating left hand movements', 'Sit to stand', 'Drawing on a paper', 
                                  'Typing on a computer keyboard', 'Assembling nuts and bolts', 
                                  'Taking a glass of water and drinking', 'Organizing sheets in a folder', 'Folding towels', 
                                  'Sitting'])
        self.taskList_Abb = np.array(['Shaking', 'Stndg', 'Wlkg', 'WlkgCnt', 'FtnR', 'FtnL', 'RamR', 'RamL', 'SitStand', 
                                      'Drwg', 'Typg', 'NtsBts', 'Drnkg', 'Sheets', 'Fldg', 'Sitng'])
        self.sessionInd = 0
        self.sessionList = np.array(['2 Weeks: Time 0', '2 Weeks: Time 30', '2 Weeks: Time 60', '2 Weeks: Time 90', 
                                     '2 Weeks: Time 120', '2 Weeks: Time 150', '1 Month'])
        self.loadedSubject = None
        self.loadedSession = None
        
        self.lastValidSubj = '1003'
        self.lastValidSess = self.sessionList[self.sessionInd]
        self.lastValidTask = self.taskList[self.taskInd]
        # Initialize variables to store offset and original timestamps
        self.offset = 0
        self.stTime_Orig = 0
        self.spTime_Orig = 0
        
        ## Create widgets
        self.plotBtn = QtGui.QPushButton('Plot Data')
        self.subjtxt = QtGui.QLineEdit(self.lastValidSubj)
        self.sesstxt = QtGui.QLineEdit(self.lastValidSess)
        self.tasktxt = QtGui.QLineEdit(self.lastValidTask)
        self.nextBtn = QtGui.QPushButton('Next Task')
        self.saveBtn = QtGui.QPushButton('Save Data')
        self.offsettxt = QtGui.QLineEdit('0')
        self.offsetBtn = QtGui.QPushButton('Set Offset')
        self.listw = QtGui.QListWidget()
        self.plot = pg.PlotWidget()
        self.stLine = pg.InfiniteLine(pen=(4,5),movable=True)
        self.spLine = pg.InfiniteLine(pen=(5,5),movable=True)

        ## Connect functions to widgets
        self.plotBtn.clicked.connect(self.plotBtn_clicked)
        self.nextBtn.clicked.connect(self.nextBtn_clicked)
        self.saveBtn.clicked.connect(self.saveBtn_clicked)
        self.offsetBtn.clicked.connect(self.offsetBtn_clicked)
        self.subjtxt.editingFinished.connect(self.subjTxt_edited)
        self.sesstxt.editingFinished.connect(self.sessTxt_edited)
        self.tasktxt.editingFinished.connect(self.taskTxt_edited)
        self.offsettxt.editingFinished.connect(self.offsetTxt_edited)
        self.listw.itemDoubleClicked.connect(self.itemClick)
        
        ## Add list of incomplete subjects to list widget
        self.refreshList()

        ## Create a grid layout to manage the widgets size and position
        layout = QtGui.QGridLayout()

        ## Add widgets to the layout in their proper positions
        layout.addWidget(self.plotBtn, 0, 0)
        layout.addWidget(self.subjtxt, 1, 0)
        layout.addWidget(self.sesstxt, 2, 0)
        layout.addWidget(self.tasktxt, 3, 0)
        layout.addWidget(self.nextBtn, 4, 0)
        layout.addWidget(self.saveBtn, 5, 0)
        layout.addWidget(self.offsettxt, 6, 0)
        layout.addWidget(self.offsetBtn, 7, 0)
        layout.addWidget(self.listw, 8, 0)  # list widget goes in bottom-left
        layout.addWidget(self.plot, 0, 1, 9, 1)  # plot goes on right side

        self.setLayout(layout)

### Functions for widgets
        
    def plotBtn_clicked(self):
        ##############################
        ##############################
        # Modify this part to take in timestamps from annotated files
        # pd.read_csv('//FS2.smpp.local\RTO\CIS-PD Videos\timestamp\GUI_timestamp.csv')
        ##############################
        ##############################
        
        # error on file location
#         df = pd.read_csv(os.path.join('//FS2.smpp.local\RTO\CIS-PD Videos\timestamp','GUI_timestamp.csv')
                         
        path = r'//FS2.smpp.local\RTO\CIS-PD Videos\timestamp'
#        df = pd.read_csv(os.path.join(path,'GUI_timestamp_missing4.csv'))
        df = pd.read_csv(os.path.join(path,'GUI_timestamp.csv'))
                         
        # 4 digit subject code
        subjInds = (df.subject_number == int(self.subjtxt.text()))
        # full activity name
        taskInds = (df.activity == self.tasktxt.text())
        # cycle
        cycleInds = (df.cycle == self.sessionInd)
        # start time
        stTime = df[subjInds & taskInds & cycleInds].start_utc
        # stop time
        spTime = df[subjInds & taskInds & cycleInds].stop_utc
        
        stTime = pd.Series([stTime.values[0]]).astype('datetime64[ms]').values[0]
        spTime = pd.Series([spTime.values[0]]).astype('datetime64[ms]').values[0]
        
        # Add offset
        print(self.offset)
        stTime = stTime + np.timedelta64(int(self.offset),'ms')
        spTime = spTime + np.timedelta64(int(self.offset),'ms')
        
        ########################
              
        ## Load accelerometer data if not already loaded
        if ((self.loadedSubject!=self.subjtxt.text()) | (self.loadedSession!=self.sessionInd)):
            self.ACC = pd.read_csv(os.path.join(self.DataPath,self.subjtxt.text() + '_session_' + str(self.sessionInd)+ '.csv'), 
                                   parse_dates=[4], index_col=0)
            self.ACC.sort_values(by = 'timestamp', axis = 0, inplace=True)
            self.loadedSubject = self.subjtxt.text()
            self.loadedSession = self.sessionInd
        
        ACC = self.ACC
        ACC = ACC[(ACC.timestamp < (spTime + np.timedelta64(self.WindowRange,'s'))) 
                  & (ACC.timestamp > (stTime - np.timedelta64(self.WindowRange,'s')))]
        if len(ACC)<1:
            print('No data found for time range')
            print(stTime, spTime, self.ACC.timestamp.iloc[0], self.ACC.timestamp.iloc[-1])
            return
        
        self.stTime_ACC = ACC.timestamp.values[0].astype('datetime64[ms]')
        T = (ACC.timestamp.values.astype('datetime64[ms]') - self.stTime_ACC).astype(float)
        
        self.plot.clear()
        
        self.plot.plot(T,ACC.x.values,pen=(1,5))
        self.plot.plot(T,ACC.y.values,pen=(2,5))
        self.plot.plot(T,ACC.z.values,pen=(3,5))
        
        stTime = stTime.astype('datetime64[ms]').astype(float) - ACC.timestamp.values[0].astype('datetime64[ms]').astype(float)
        spTime = spTime.astype('datetime64[ms]').astype(float) - ACC.timestamp.values[0].astype('datetime64[ms]').astype(float)
        
        self.stTime_Orig = stTime
        self.spTime_Orig = spTime
        
#         stTime = stTime + self.offset
#         spTime = spTime + self.offset
        
        self.stLine.setValue(stTime); self.plot.addItem(self.stLine, ignoreBounds=True)
        self.spLine.setValue(spTime); self.plot.addItem(self.spLine, ignoreBounds=True)
        
        self.plot.setYRange(-3,3)
        self.plot.setXRange(stTime-self.WindowRange/2*1000, self.WindowRange/2*1000+spTime)
        
        
    def nextBtn_clicked(self):
        self.taskInd = self.taskInd + 1
        self.tasktxt.setText(self.taskList[self.taskInd])
        self.lastValidTask = self.tasktxt.text()
        
        self.plotBtn_clicked()
        
    def saveBtn_clicked(self):
        stTime = np.timedelta64(int(self.stLine.value()),'ms') + self.stTime_ACC
        spTime = np.timedelta64(int(self.spLine.value()),'ms') + self.stTime_ACC
        
        ACC = self.ACC
        ACC = ACC[(ACC.timestamp < (spTime)) & (ACC.timestamp > (stTime))]
        filepath = os.path.join(self.SavePath,
                                self.subjtxt.text()+'_'+str(self.sessionInd)+'_'+self.taskList_Abb[self.taskInd]+'.csv')
        ACC.to_csv(path_or_buf = filepath, index=False)
        ## Refresh list of files to annotate
        self.refreshList()

    def offsetBtn_clicked(self):
        self.offset = self.offset + np.mean([self.stLine.value()-self.stTime_Orig, self.spLine.value()-self.spTime_Orig])
        self.offsettxt.setText(str(self.offset/1000))
        self.offsetTxt_edited()
        
    ## Text Box Functions - Check for valid values and replace if not
    def subjTxt_edited(self):
        if sum(self.subjList == self.subjtxt.text())<1:
            print('Not a valid subject')
            self.subjtxt.setText(self.lastValidSubj)
        else:
            self.lastValidSubj = self.subjtxt.text()
    
    def sessTxt_edited(self):
        self.sessionInd = np.argmax(self.sessionList == self.sesstxt.text())
        if sum(self.sessionList == self.sesstxt.text())<1:
            print('Not a valid session')
            self.sesstxt.setText(self.lastValidSess)
        else:
            self.lastValidSess = self.sesstxt.text()

        
    def taskTxt_edited(self):
        self.taskInd = np.argmax(self.taskList == self.tasktxt.text())
        if sum(self.taskList == self.tasktxt.text())<1:
            print('Not a valid task')
            self.tasktxt.setText(self.lastValidTask)
        else:
            self.lastValidTask = self.tasktxt.text()
            
    def offsetTxt_edited(self):
        self.offset = float(self.offsettxt.text())*1000
    
    ## List Widget Functions
    def itemClick(self, item):
        self.subjtxt.setText(item.text()[0:4])
        self.sessInd = int(item.text()[5:6])
        self.sesstxt.setText(self.sessionList[self.sessInd])
        self.taskInd = np.argmax(item.text()[7:]==self.taskList_Abb)
        self.tasktxt.setText(self.taskList[self.taskInd])
        
        self.sessTxt_edited()
        self.subjTxt_edited()
        self.taskTxt_edited()
        
            
### Auxillary functions
    
    def getTimeDelta(self, ts): #Convert string stopwatch time to timedelta
        return np.timedelta64(int(ts[0:2]),'h') + np.timedelta64(int(ts[3:5]),'m') + np.timedelta64(int(ts[6:]),'s')
        
    def refreshList(self):
        self.listw.clear()
        for subj, sess, task in product(self.subjList, list(range(7)), self.taskList_Abb):
            filename = subj+'_'+str(sess)+'_'+task
            if not os.path.isfile(os.path.join(self.SavePath,filename+'.csv')):
                if os.path.isfile(os.path.join(self.DataPath,subj+'_session_'+str(sess)+'.csv')):
                    self.listw.addItem(filename)
    

In [ ]:
window = AccViewer()
window.show()
app.exec_()

0
No data found for time range
2017-07-03T21:26:07.380000000 2017-07-03T21:26:32.972000000 2017-07-03 15:20:00.017000 2017-07-03 16:03:51.522000
-21800000.0


In [23]:
np.timedelta64(int(-20506.43),'ms')

numpy.timedelta64(-20506,'ms')

In [ ]:
pd.DataFrame.drop()

In [22]:
pd.Series([1.499122e+12]).astype('datetime64[ms]').values[0]

numpy.datetime64('2017-07-03T22:46:40.000000000')

In [10]:
path = r'//FS2.smpp.local\RTO\CIS-PD Videos\timestamp'
df = pd.read_csv(os.path.join(path,'GUI_timestamp.csv'))

In [11]:
df.head(5)

,Unnamed: 0,UTC_create_date,subj_cycle,subject code,start frame,stop frame,activity,cycle,shortname,start time sec,stop time sec,start_utc,stop_utc,subject_number
0,0,1501681699000,cisnwh81,cisnwh8,1118.0,2145.0,Standing,0.0,Stndg,37.0,71.0,1.501682e+12,1.501682e+12,1030
1,1,1501681699000,cisnwh81,cisnwh8,2315.0,3333.0,Walking,0.0,Wlkg,77.0,111.0,1.501682e+12,1.501682e+12,1030
2,2,1501681699000,cisnwh81,cisnwh8,3608.0,4800.0,Walking while counting,0.0,WlkgCnt,120.0,160.0,1.501682e+12,1.501682e+12,1030
3,3,1501681699000,cisnwh81,cisnwh8,5832.0,6518.0,Finger to nose--right hand,0.0,FtnR,194.0,217.0,1.501682e+12,1.501682e+12,1030
4,4,1501681699000,cisnwh81,cisnwh8,6518.0,7121.0,Finger to nose--left hand,0.0,FtnL,217.0,237.0,1.501682e+12,1.501682e+12,1030


# Reference info

Project status

Completed
1019

In progress
1003 (cycle 0-5 / time 0,30,60,90,120,150) offset -21800
- Error: saved Standing, Walking, Walk with count to cycle 2. Can I just save over in this situation?
- Not clear on stand, walk, and walk with count in cycle 1
1005 (cycle 0-3 / time 0,30,60,90) offset -20420
- Error: saved first cycle under '2 Weeks: Time 90' with offset -20929.32926742833
1007 (cycle 0-3/ time 0,30,60,90) offset -21940.197417575815
- unclear on activities
1009 (cycle 0,3,5 / time 0,90,150)
- offset -19000 doesn't look right
1023 - skip UAB
1024 (cycle 2 / time 60)
- no offset, but walk vs walk and count not very clear
1038 ???
1039 - skip UAB
1043 - skip UAB
1048 - skip cisrol12
1050 (cycle 1-5 / time 30,60,90,120,150) offset -20800
- unclear on activities


Steps to complete annotation:
Load GUI
Set subj number
Set time: 90
Activity: Standing
Look for standing and walking activities
Set offset
Check start/stop, adjust
Save Data
Next Task - don't modify NtsBts activity

Note on cycle/time
cycle 0 / time 0
cycle 1 / time 30
cycle 2 / time 60
cycle 3 / time 90
cycle 4 / time 120

Summary of data that is off
- 1003 is off by 1 yr and 5 hours and some seconds, cycle 3 was off by an additional 9 min
- 1030 is off by several seconds (usually around 30 sec)
- 1005 is off by 1 yr 5 hrs and some seconds, cycle 2 is off by an additional 5 min
- 1007 is off by 1 yr 5 hrs and some seconds
- 1009 is off by 1 yr 5 hrs and some seconds
- 1019 is off by 29.5 min
- 1024 is off by 50 sec
- 1048 cycle 1 3.5 hours off, cycle 2 missing, cycle 4 is 4 hrs off, cycle 5 is about 4 hrs off
- 1050 is off by 1 yr, 5 hrs and some sec

These videos we suspect editing, so Skip these subjects (ciscid4, ciscih8, ciccij10):
- 1023 is off by 18 days, but the watch shaking time for all cycles the same
- 1039 is off by 2 months, 13 days, and variable time but the watch shaking time for all cycles the same
- 1043 is off by 2 months, 1 day, but  the watch shaking time for all cycles the same